<a href="https://colab.research.google.com/github/antonychackotc/nlp-framework/blob/main/nlp_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%%writefile app1.py

Overwriting app1.py


In [11]:
!pip install -q streamlit
!pip install -q pyngrok
!pip install -q localtunnel

ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [12]:
pip install streamlit pandas numpy nltk wordcloud matplotlib scikit-learn seaborn spacy textblob langdetect googletrans==4.0.0-rc1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [ ]:
# creat nlp application using streamlit

# upload a dataset -( csv, excell,text,json)
# analysis a dataset and tell about dataset using word cloud

# create a sepearate tab for each analysis

# 1) Text Preprocessing - seperate tabs
# - sub tabs
# a)Tokenization shows ( sentense tokenisation, word tokenisation, letter tokenisation)
# b)stop words removal
# c)lowercase
# d) remove punchuation and special characters
# e) lowercasing
# f) lemattization

# 2) Feature Extraction (Vectorisation) - seperate tabs
# -sub tabs
# a) Bag of words
# b)TF-IDF
# TF - Term Frequency ( words repeated)
# IDF - Inverse Document Frequency ( Log Value Check which Value was less importance that value was removed)
# c)word embedding (Assuming Propabalities values - relationship)
# d) n- grams (unigrams,bigrams,trigrams)
# e) parts of speech

# 3) model train - seperate tabs
# choose algorithm
# - Navie bayes classifier
# - svm
# - decession tree
# - random forest

# show multi dimensional chart pca

# model evaluation
# -show loading time

# show both train and test
# -accuracy
# -precession
# -recall
# -f1 score
# show confusion matrix chart and table

# download train and test data

# 4) Future Prediction - seperate tabs

# detect this data based on

# if ham or spam based means show type the content or email show spam or ham
# else content show sentiment analysis
# else show topics prediction based on summary
# else - language detection
# else - Document classification

# show all in one



In [13]:
# Step 1: Uninstall NLTK and delete its data entirely
!pip uninstall -y nltk
!rm -rf /root/nltk_data
!rm -rf /usr/local/nltk_data

# Step 2: Reinstall a stable version of NLTK
!pip install nltk==3.8.1  # Pin to a stable version

# Step 3: Re-download necessary datasets
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Step 4: Print data path to verify
print(nltk.data.path)


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...


['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [21]:
%%writefile app1.py
import streamlit as st
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
from textblob import TextBlob
from langdetect import detect

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Function to load data
def load_data(uploaded_file):
    if uploaded_file.name.endswith('.csv'):
        data = pd.read_csv(uploaded_file)
    elif uploaded_file.name.endswith('.xlsx'):
        data = pd.read_excel(uploaded_file)
    elif uploaded_file.name.endswith('.json'):
        data = pd.read_json(uploaded_file)
    else:
        data = pd.read_csv(uploaded_file, sep='\t')
    return data

# Function to generate word cloud
def generate_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    st.pyplot(plt)

# Load spaCy model only when needed
@st.cache(allow_output_mutation=True)
def load_spacy_model():
    import spacy
    return spacy.load('en_core_web_sm')

# Load GPT-2 model and tokenizer only when needed
@st.cache(allow_output_mutation=True)
def load_gpt2_model():
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    return tokenizer, model

# Load translator only when needed
@st.cache(allow_output_mutation=True)
def load_translator():
    from googletrans import Translator
    return Translator()

# Streamlit app
def main():
    st.title("NLP Application")

    # Initialize session state
    if 'model' not in st.session_state:
        st.session_state.model = None
    if 'tfidf_vectorizer' not in st.session_state:
        st.session_state.tfidf_vectorizer = None
    if 'X_tfidf' not in st.session_state:
        st.session_state.X_tfidf = None

    # Upload dataset
    uploaded_file = st.file_uploader("Upload your dataset (CSV, Excel, JSON, Text)", type=["csv", "xlsx", "json", "txt"])
    if uploaded_file is not None:
        data = load_data(uploaded_file)
        st.write("Dataset Overview:")
        st.write(data.head())

        # Select feature and target columns
        st.subheader("Select Feature and Target Columns")
        feature_column = st.multiselect("Select Feature Column", data.columns)
        target_column = st.selectbox("Select Target Column", data.columns)

        # Word Cloud
        st.subheader("Word Cloud")
        if feature_column:
            combined_text = ' '.join(data[feature_column].apply(lambda x: ' '.join(x.astype(str)), axis=1))
            generate_wordcloud(combined_text)
        else:
            st.warning("Please select at least one feature column to generate the word cloud.")

        # Tabs for different analyses
        tab1, tab2, tab3, tab4 = st.tabs(["Text Preprocessing", "Feature Extraction", "Model Training", "Future Prediction"])

        with tab1:
            st.header("Text Preprocessing")
            sub_tab1, sub_tab2, sub_tab3, sub_tab4, sub_tab5, sub_tab6 = st.tabs(["Tokenization", "Stop Words Removal", "Lowercasing", "Remove Punctuation", "Lemmatization", "Lowercasing"])

            with sub_tab1:
                st.subheader("Tokenization")
                st.write("Sentence Tokenization:")
                sentences = sent_tokenize(combined_text)
                st.write(sentences)

                st.write("Word Tokenization:")
                words = word_tokenize(combined_text)
                st.write(words)

                st.write("Letter Tokenization:")
                letters = list(combined_text)
                st.write(letters)

            with sub_tab2:
                st.subheader("Stop Words Removal")
                stop_words = set(stopwords.words('english'))
                filtered_words = [word for word in words if word.lower() not in stop_words]
                st.write(filtered_words)

            with sub_tab3:
                st.subheader("Lowercasing")
                lowercased_words = [word.lower() for word in words]
                st.write(lowercased_words)

            with sub_tab4:
                st.subheader("Remove Punctuation and Special Characters")
                import string
                cleaned_words = [word for word in words if word.isalnum()]
                st.write(cleaned_words)

            with sub_tab5:
                st.subheader("Lemmatization")
                lemmatizer = WordNetLemmatizer()
                lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
                st.write(lemmatized_words)

            with sub_tab6:
                st.subheader("Lowercasing")
                lowercased_words = [word.lower() for word in words]
                st.write(lowercased_words)

        with tab2:
            st.header("Feature Extraction")
            sub_tab1, sub_tab2, sub_tab3, sub_tab4, sub_tab5 = st.tabs(["Bag of Words", "TF-IDF", "Word Embedding", "N-grams", "Parts of Speech"])

            with sub_tab1:
                st.subheader("Bag of Words")
                vectorizer = CountVectorizer()
                X = vectorizer.fit_transform(data[feature_column])
                st.write(X.toarray())

            with sub_tab2:
                st.subheader("TF-IDF")
                if feature_column:
                    combined_text_series = data[feature_column].apply(lambda x: ' '.join(x.astype(str)), axis=1)
                    tfidf_vectorizer = TfidfVectorizer()
                    X_tfidf = tfidf_vectorizer.fit_transform(combined_text_series)
                    st.write(X_tfidf.toarray())
                    st.session_state.tfidf_vectorizer = tfidf_vectorizer
                    st.session_state.X_tfidf = X_tfidf
                else:
                    st.warning("Please select at least one feature column.")

            with sub_tab3:
                st.subheader("Word Embedding")
                nlp = load_spacy_model()
                doc = nlp(combined_text)
                st.write([token.vector for token in doc])

            with sub_tab4:
                st.subheader("N-grams")
                st.write("Unigrams:")
                unigrams = list(nltk.ngrams(words, 1))
                st.write(unigrams)

                st.write("Bigrams:")
                bigrams = list(nltk.ngrams(words, 2))
                st.write(bigrams)

                st.write("Trigrams:")
                trigrams = list(nltk.ngrams(words, 3))
                st.write(trigrams)

            with sub_tab5:
                st.subheader("Parts of Speech")
                nlp = load_spacy_model()
                doc = nlp(combined_text)
                st.write([(token.text, token.pos_) for token in doc])

        with tab3:
            st.header("Model Training")
            st.subheader("Choose Algorithm")
            algorithm = st.selectbox("Select Algorithm", ["Naive Bayes", "SVM", "Decision Tree", "Random Forest", "Gradient Boosting"])

            # Cross-validation options
            st.subheader("Cross-Validation")
            cv_method = st.selectbox("Select Cross-Validation Method", ["None", "K-Fold", "Stratified K-Fold", "Hold Out"])
            if cv_method != "None":
                cv_folds = st.number_input("Number of Folds", min_value=2, max_value=10, value=5)

            # Hyperparameter tuning options
            st.subheader("Hyperparameter Tuning")
            tuning_method = st.selectbox("Select Tuning Method", ["None", "Grid Search", "Random Search"])

            if st.button("Train Model"):
                y = data[target_column]
                if st.session_state.X_tfidf.shape[0] != data[target_column].shape[0]:
                    st.error(f"Number of samples in X_tfidf ({st.session_state.X_tfidf.shape[0]}) and y ({data[target_column].shape[0]}) do not match. Please check your data.")
                else:
                    X_train, X_test, y_train, y_test = train_test_split(st.session_state.X_tfidf, data[target_column], test_size=0.2, random_state=42)

                if algorithm == "Naive Bayes":
                    model = MultinomialNB()
                elif algorithm == "SVM":
                    model = SVC()
                elif algorithm == "Decision Tree":
                    model = DecisionTreeClassifier()
                elif algorithm == "Random Forest":
                    model = RandomForestClassifier()
                elif algorithm == "Gradient Boosting":
                    model = GradientBoostingClassifier()

                # Hyperparameter tuning
                if tuning_method == "Grid Search":
                    if algorithm == "Random Forest":
                        param_grid = {
                            'n_estimators': [100, 200, 300],
                            'max_depth': [None, 10, 20],
                            'min_samples_split': [2, 5, 10]
                        }
                        grid_search = GridSearchCV(model, param_grid, cv=cv_folds if cv_method != "None" else 5)
                        grid_search.fit(X_train, y_train)
                        model = grid_search.best_estimator_
                        st.write("Best Parameters:", grid_search.best_params_)
                elif tuning_method == "Random Search":
                    if algorithm == "Random Forest":
                        param_dist = {
                            'n_estimators': [100, 200, 300],
                            'max_depth': [None, 10, 20],
                            'min_samples_split': [2, 5, 10]
                        }
                        random_search = RandomizedSearchCV(model, param_dist, n_iter=10, cv=cv_folds if cv_method != "None" else 5)
                        random_search.fit(X_train, y_train)
                        model = random_search.best_estimator_
                        st.write("Best Parameters:", random_search.best_params_)

                # Cross-validation
                if cv_method == "K-Fold":
                    cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds)
                    st.write(f"Cross-Validation Accuracy: {cv_scores.mean():.2f} (±{cv_scores.std():.2f})")
                elif cv_method == "Stratified K-Fold":
                    skf = StratifiedKFold(n_splits=cv_folds)
                    cv_scores = cross_val_score(model, X_train, y_train, cv=skf)
                    st.write(f"Stratified K-Fold Accuracy: {cv_scores.mean():.2f} (±{cv_scores.std():.2f})")

                # Train the model
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                st.subheader("Model Evaluation")
                st.write("Accuracy:", accuracy_score(y_test, y_pred))
                st.write("Precision:", precision_score(y_test, y_pred, average='weighted'))
                st.write("Recall:", recall_score(y_test, y_pred, average='weighted'))
                st.write("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

                st.subheader("Confusion Matrix")
                cm = confusion_matrix(y_test, y_pred)
                plt.figure(figsize=(8, 6))
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
                plt.xlabel('Predicted')
                plt.ylabel('Actual')
                plt.title('Confusion Matrix')
                st.pyplot(plt)

                st.subheader("Confusion Matrix Explanation")
                st.write("""
                - **True Positive (TP)**: The model correctly predicted the positive class.
                - **False Positive (FP)**: The model incorrectly predicted the positive class (Type I Error).
                - **True Negative (TN)**: The model correctly predicted the negative class.
                - **False Negative (FN)**: The model incorrectly predicted the negative class (Type II Error).
                """)
                st.write(f"""
                - **True Positives (TP)**: {cm[1, 1]}
                - **False Positives (FP)**: {cm[0, 1]}
                - **True Negatives (TN)**: {cm[0, 0]}
                - **False Negatives (FN)**: {cm[1, 0]}
                """)

                # Store the trained model in session state
                st.session_state.model = model

        with tab4:
            st.header("Future Prediction")
            st.subheader("Select Prediction Type")
            prediction_type = st.selectbox("Select Prediction Type", [
                "Spam/Ham Detection",
                "Sentiment Analysis",
                "Topic Prediction",
                "Document Classification",
                "Language Detection",
                "Translate to Tamil",
                "Text Autogeneration"
            ])

            input_text = st.text_area("Input Text")

            if st.button("Predict"):
                if st.session_state.model is None:
                    st.warning("Please train a model in the 'Model Training' tab first.")
                else:
                    if prediction_type == "Spam/Ham Detection":
                        input_vector = st.session_state.tfidf_vectorizer.transform([input_text])
                        prediction = st.session_state.model.predict(input_vector)
                        st.write("Prediction:", "Spam" if prediction[0] == 1 else "Ham")

                    elif prediction_type == "Sentiment Analysis":
                        blob = TextBlob(input_text)
                        sentiment = blob.sentiment
                        st.write("Sentiment Polarity:", sentiment.polarity)
                        st.write("Sentiment Subjectivity:", sentiment.subjectivity)
                        st.write("Sentiment:", "Positive" if sentiment.polarity > 0 else "Negative" if sentiment.polarity < 0 else "Neutral")

                    elif prediction_type == "Topic Prediction":
                        lda = LatentDirichletAllocation(n_components=5, random_state=42)
                        lda.fit(st.session_state.X_tfidf)
                        input_vector = st.session_state.tfidf_vectorizer.transform([input_text])
                        topic_distribution = lda.transform(input_vector)
                        st.write("Topic Distribution:", topic_distribution)
                        st.write("Predicted Topic:", np.argmax(topic_distribution))

                    elif prediction_type == "Document Classification":
                        input_vector = st.session_state.tfidf_vectorizer.transform([input_text])
                        prediction = st.session_state.model.predict(input_vector)
                        st.write("Predicted Class:", prediction[0])

                    elif prediction_type == "Language Detection":
                        language = detect(input_text)
                        st.write("Detected Language:", language)

                    elif prediction_type == "Translate to Tamil":
                        translator = load_translator()
                        translation = translator.translate(input_text, dest='ta')
                        st.write("Translated Text:", translation.text)

                    elif prediction_type == "Text Autogeneration":
                        tokenizer, model = load_gpt2_model()
                        inputs = tokenizer.encode(input_text, return_tensors="pt")
                        outputs = model.generate(inputs, max_length=200, num_return_sequences=1, temperature=0.7)
                        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                        st.write("Generated Text:", generated_text)

if __name__ == "__main__":
    main()

Overwriting app1.py


In [22]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token("2rI2XurhgC2fxlYDtteHntWpCJf_5b1kDx2SLmwgq8GukDEyc")

# Run the Streamlit app in the background
!streamlit run app1.py &>/dev/null&

# Create a public URL using ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is running at {public_url}")
except Exception as e:
    print(f"Error: {e}")
    print("Trying to run with localtunnel")
    !streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501

Streamlit app is running at NgrokTunnel: "https://816a-34-80-204-3.ngrok-free.app" -> "http://localhost:8501"
